In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from PIL import Image
from copy import deepcopy

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# 1. Load and display image
image = Image.open('image.png').convert('L')  # Grayscale
img_array = np.array(image)
plt.imshow(img_array, cmap='gray')
plt.title("Original Image")
plt.axis('off')
plt.show()

In [ ]:
image.size          # Get the size of the image

In [ ]:
img_array.size      # Get the shape of the image array (height, width)

In [ ]:
# 2. Plot histogram and cumulative histogram
hist, bins = np.histogram(img_array.flatten(), bins=256, range=[0,256])
cum_hist = np.cumsum(hist)

# Plot side by side
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

# Plot histogram
axs[0].bar(range(256), hist, color='blue')
axs[0].set_title('Histogram')
axs[0].set_xlabel('Intensity Value')
axs[0].set_ylabel('Frequency')

# Plot cumulative histogram
axs[1].plot(range(256), cum_hist, color='red')
axs[1].set_title('Cumulative Histogram')
axs[1].set_xlabel('Intensity Value')
axs[1].set_ylabel('Cumulative Count')

plt.tight_layout()
plt.show()

In [ ]:
# 3. Quantization by reducing bit-depth (e.g., to 2 bits, 4 levels)
def quantize_image(img, bits):
    levels = 2 ** bits         # Total number of intensity levels for the given bit-depth
    factor = 256 // levels     # Size of each quantization step
    quantized = (img // factor) * factor    # Reduce bit-depth: divide by factor (integer division), then multiply to scale back
                                            # This maps pixel values to their nearest lower quantization level
    return quantized

quant_img = quantize_image(img_array, 2)      ### Play with this value to see different quantization levels
plt.imshow(quant_img, cmap='gray')
plt.title('Quantized Image (2 bits)')
plt.axis('off')
plt.show()

In [ ]:
# 4. K-means clustering for k representative colors
def kmeans_quantization(img, k):
    flat = img.flatten().reshape(-1, 1)
    
    # Initialize and fit K-means
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=0)         
    kmeans.fit(flat)
    
    # Replace each pixel with its cluster center
    clustered = kmeans.cluster_centers_[kmeans.labels_].reshape(img.shape)
    return clustered.astype(np.uint8)

kmeans_img = kmeans_quantization(img_array, 2)     ### Play with k value
plt.imshow(kmeans_img, cmap='gray')
plt.title('K-means Quantized (k=2)')
plt.axis('off')
plt.show()

In [ ]:
# 5. Weighted Mean Calculation
def weighted_mean(hist, bins):
    return np.sum(hist * bins[:-1]) / np.sum(hist)

mean_val = weighted_mean(hist, bins)
print(f'Weighted Mean Intensity: {mean_val:.2f}')

In [ ]:
# 6. Multi-level Quantization with boundary adjustment
def multi_level_quantization(img, k, max_iters=100):
    lower = 0
    upper = 255
    boundaries = np.linspace(lower, upper, k+1)         # Initialize boundaries evenly spaced between 0 and 255
    img_flat = img.flatten()
    prev_mse = None
    
    for iteration in range(max_iters):
        means = []
        
        # Step 1: Compute the mean of pixel values in each bin
        for i in range(k):
            # Create a boolean mask for the current bin
            # The mask is a boolean array that marks which pixels fall within the current bin [boundaries[i], boundaries[i+1])
            # For example, mask[j] = True if img_flat[j] is inside the i-th bin, False otherwise           
            mask = (img_flat >= boundaries[i]) & (img_flat < boundaries[i+1])
            if np.sum(mask) == 0:

                # If no pixels fall in this bin, use the midpoint as the mean
                mean = (boundaries[i] + boundaries[i+1]) / 2
            else:
                # Otherwise compute the mean of the pixels in this bin
                mean = np.mean(img_flat[mask])
            means.append(mean)

        # Step 2: Adjust internal boundaries based on means
        new_boundaries = deepcopy(boundaries)
        for i in range(1, k):
            new_boundaries[i] = (means[i-1] + means[i]) / 2
        boundaries = new_boundaries

        # Step 3: Quantize the image
        quantized_img = np.zeros_like(img_flat)
        for i in range(k):
            # Apply the same mask logic to assign mean to each pixel
            mask = (img_flat >= boundaries[i]) & (img_flat < boundaries[i+1])
            quantized_img[mask] = means[i]
        quantized_img = quantized_img.reshape(img.shape)

        # Step 4: Compute mean squared error
        mse = np.sqrt(np.mean((img - quantized_img) ** 2))
        print(f"Iteration {iteration+1}: MSE = {mse:.2f}")

        # Step 5: Check for convergence
        if prev_mse is not None and abs(prev_mse - mse) < 0.01:
            break
        prev_mse = mse

    return quantized_img.astype(np.uint8)

multi_img = multi_level_quantization(img_array, 2)    ### Play with k value
plt.imshow(multi_img, cmap='gray')
plt.title('Multi-level Quantized Image')
plt.axis('off')
plt.show()